In [ ]:
import socket
import threading
import cv2 as cv
import numpy as np

## Server program to which Client will connect

In [ ]:
soc = socket.socket()
soc.bind(("", 5678)) #here localhost IP is binded with the port number 5678
soc.listen()
session, address = soc.accept() #here it will accepts the request from anywhere
print(session.recv(2046)) 
camIndex = 'http://192.168.0.103:8080/video' # here this IP is of IPCAMERA app 
cam = cv.VideoCapture(camIndex) # here camera is started

def sender():
    while True:
        ret, photo = cam.read()
        photo = cv.resize(photo, (640, 480))
        print(photo.shape)
        if ret:
            session.send(np.ndarray.tobytes(photo))
        else: print("No Frame Found")

def receiver():
    framesLost = 0
    print("Connecton Established")
    while True:
        framesLost += 1 
        data = session.recv(100000000)
        if(data == b'finished'):
            print("Finished")
            cam.release()
            session.close()
        else:
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3:
                cv.imshow('Hi I am Client', photo.reshape(480, 640, 3))
                if cv.waitKey(1) == 13:
                    session.send(b'finished')
                    cam.release()
                    cv.destroyAllWindows()
                    break
            else:
                print("Lost {} frames".format(framesLost) )
                
threading.Thread(target=sender).start()
threading.Thread(target=receiver).start()